In [5]:
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display, HTML, IFrame
from ipywidgets import interact,fixed
import pandas as pd
from mpl_toolkits import mplot3d
from mpl_toolkits.mplot3d import axes3d
from matplotlib.patches import FancyArrowPatch
from mpl_toolkits.mplot3d import proj3d

plt.rcParams["figure.figsize"] = [12, 9]

from numpy.linalg import norm
from numpy import cos,sin,tan,arctan,exp,log,pi,sqrt

$\newcommand{\RR}{\mathbb{R}}$
$\newcommand{\bv}[1]{\begin{bmatrix} #1 \end{bmatrix}}$
$\renewcommand{\vec}{\mathbf}$


## Announcements

  - Quiz 4 in recitation this week. 
    - partial derivatives
  - Homework 5 posted, due 10/08
  - First midterm exam - Thur 10/10
    - No quiz that week!
    - review material in Canvas under "Files $\to$ practice"
    - Review sessions TBA.
    

# One-minute Review

The **linearization** of a differential function $f(x,y,z)$ about a point $(a,b,c)$ is the linear function $$L(x,y,z) = f(a,b,c) + f_x(a,b,c)(x-a) + f_y(a,b,c)(y-b) + f_z(a,b,c)(z-c).$$

Its graph is tangent to the graph of $f$. 

# Lecture 10

  - Objectives
    - Compute derivatives of compositions.
    - Compute derivatives from implicit relations.
    - Review for midterm.
    
  - Resources
    - Content
      - Stewart: §14.5
      - New Strang: [§4.5](https://cnx.org/contents/oxzXkyFi@6.1:8QoqMp7K@7/4-5-The-Chain-Rule)
      - [Slides](https://hub.columbiajupyter2.org/hub/user-redirect/git-pull?repo=https%3A%2F%2Fgithub.com%2Fdrewyoungren%2Fmvc-f19&urlpath=tree%2Fmvc-f19%2Fslides%2Fmvc-L10.ipynb) via JupyterHub
    - Practice
      - Mooculus: [Chain Rule](https://ximera.osu.edu/mooculus/calculus3/directionalDerivativeAndChainRule/digInChainRule)
    - Extras
      - CalcBLUE: WARNING! This is very matrix-dependent (and thus extra) [The Chain Rule](https://youtu.be/3WtjD3bewBk) 

## Theorem of Differentiability

If all partial derivatives $\frac{\partial f}{\partial x_i}$ are continuous in a neighborhood of a point $P$, then $f$ is diffferentiable at $P$. 

#### Nonexample

Consider $$f(x,y) = \frac{xy}{x^2 + y^2}.$$

# The Chain Rule

Start by recalling the 1-D chain rule $$(f\circ g)'(x) = f'(g(x))g'(x)$$

or if $u=g(x)$, $$ \frac{df}{dx} = \frac{df}{du}\frac{du}{dx}$$ where the derivatives are evaluated at the appropriate point.

## The main problem

Suppose a hiking party charts a route on a map given by $\langle x(t),y(t) \rangle$ where the altitude is given by a function $f(x,y)$.

What is $\frac{df}{dt}$ at a given time $t_0$?

In [25]:
f = lambda x,y: exp(-4*(y-sin(x))**2)*(1-np.abs(x+pi/2)/5)

@interact
def _(func=fixed(f),angle=(-90,120,6),vangle=(0,90,6),pt=(1,98)):
    pt = 99-pt
    fig = plt.figure(figsize = (12,6))
    ax = fig.add_subplot(121,projection='3d')
    ax.view_init(vangle,angle)
    for c in 'xyz':
#         getattr(ax,f"set_{c}lim")([-1,1]);    
        getattr(ax,f"set_{c}label")(f"${c}$",size=16)
    x =  np.linspace(-4,4,601)
    y =  np.linspace(-2,2,301)    
    X,Y = np.meshgrid(x,y)
    Z = func(X,Y)
    ax.plot_surface(X,Y,Z,alpha=.6,cmap='ocean',rcount=100,ccount=100);
    t = np.linspace(-pi/2,pi/2,100)
    X = t
    Y = t/pi - 1/2 - t*(t**2-pi**2/4)/5
    Z = func(X,Y)
    ax.plot(X,Y,Z,lw=6,color='r',alpha=1)
    ax2 = fig.add_subplot(122)
    ax2.plot((pi/2-t)/pi*4,Z)
    ax2.set_xlabel("$t$",size=16)
    ax2.set_ylabel("$z$",size=16)
    dt = t[pt+1]-t[pt-1]
    dx = X[pt+1]-X[pt-1]
    dy = Y[pt+1]-Y[pt-1]
    dz = Z[pt+1]-Z[pt-1]
    s = np.linspace(-20,20,100)
    ax.plot(X[pt] + dx*s,Y[pt] + dy*s,Z[pt] + dz*s,lw=4)
    ax.set_zlim3d([0,1])
    ax2.plot((pi/2-t)/pi*4,dz/dt*(t-t[pt])+Z[pt])
    ax2.set_ylim([0,1])

interactive(children=(IntSlider(value=12, description='angle', max=120, min=-90, step=6), IntSlider(value=42, …

##### Clue: replace $f$ with $L$

In [35]:
@interact
def _(angle=(-100,14,6)):
    fig = plt.figure()
    ax = fig.add_subplot(111,projection='3d')
    x = y = np.linspace(0,2,75)
    x,y = np.meshgrid(x,y)
    ax.view_init(30,angle)
    for c in 'xyz':
    #         getattr(ax,f"set_{c}lim")([-1,1]);    
        getattr(ax,f"set_{c}label")(f"${c}$",size=16)
    ax.plot_surface(x,y,-(x-2)**2 - (y-2)**2/2,alpha=.5)
    ax.plot_surface(x,y,2*(x-1) + (y-1) -3/2,alpha=.5)
    t = np.linspace(0,2,50)
    ax.plot(t,t,-(t-2)**2 - (t-2)**2/2)
    ax.plot(t,t,2*(t-1) + (t-1) - 3/2);

interactive(children=(IntSlider(value=-46, description='angle', max=14, min=-100, step=6), Output()), _dom_cla…

That is, linearize $f$ around the point $a,b) = (x(t_0),y(t_0)$.

$$\frac{d}{dt} f(x(t),y(t)) = \frac{d}{dt} L(x(t),y(t))$$

$$= \frac{d}{dt} \left( f(a,b) + f_x(a,b)(x(t)-a) + f_y(a,b)(y(t) - b)\right)$$

$$= f_x(a,b) x'(t) + f_y(a,b) y'(t)$$

$$= \frac{\partial f}{\partial x}\frac{dx}{dt} + \frac{\partial f}{\partial y}\frac{dy}{dt}$$ where, again, we're careful to evaluate the partials at the appropriate point.

# The Chain Rule

If $f(x_1,\ldots,x_n)$ is differentiable at $\vec x_0 = \langle x_1(t_0),\ldots,x_n(t_0) \rangle$ and each $x_i$ is differetiable at $t_0$, then 

$$ \frac {df}{dt}\Big|_{t_0} = \sum_{i=1}^n \frac{\partial f}{\partial x_1}\Big|_{\vec x_0}\frac{dx}{dt}\Big|_{t_0} $$

#### Definition

This is more easily expressed using the **gradient** vector of a function $$\nabla f = \begin{bmatrix} f_{x_1} \\ f_{x_2} \\ \vdots \\ f_{x_n} \end{bmatrix}$$

*There will be **much** more to say about this later; for now, just treat it as notation.*

Thus for differetiable functions $f:\RR^n\to \RR$ and $\vec r: \RR \to \RR^n$, the composition has derivative $$\frac{d}{dt} f(\vec r(t)) = \nabla f(\vec r(t))\cdot \vec r'(t).$$

## Other forms

Other cases can be handled by extrapolatig from the case above. 

$$\frac{\partial}{\partial t} f(x(s,t),y(s,t),z(s,t)) = \frac{\partial f}{\partial x} \frac{\partial x}{\partial t} + \frac{\partial f}{\partial y}\frac{\partial y}{\partial t} +\frac{\partial f}{\partial z}\frac{\partial z}{\partial t} $$


### Example

A particle circles the $z$-axis every second at a distance of 1 unit. If it is confined to the surface $z=xy$, find the maximum of $\frac{dz}{dt}$.

### Exercises

  1. ![Wheat problem from Stewart](wheat-problem.png)

  2. Suppose 
\begin{align} 
z &= z(x,y) \\
x &= x(u,v) \\
y &= y(u,v) \\
u &= u(s,t) \\
v &= v(s,t) \\
\end{align}
are all differetiable. Find an expression for $\frac{\partial z}{\partial s}$.

## Example

If $g:\RR\to\RR$ is any smooth function, show that $f(x,y) = g(x^2+y^2)$ is radially symmetric. That is, $\frac{\partial f}{\partial \theta} =0$

# Implicit differentiation

Everything  implicit relationship among variables $x$ and $y$ ca nbe expressed as a level set $$ F(x,y) = c. $$

To find the rae of change $\frac{dy}{dx}$ determined by this relation, we ca nbe clever and differenntiate the whole expressio nwith respect to $x$. 

$$ \frac{d}{dx}F(x,y) = 0$$

**WARNING** Mind your $d$'s and $\partial$'s.

$$ \frac{d}{dx}F(x,y(x)) = 0$$

$$ \frac{\partial F}{\partial x}\frac{dx}{dx} + \frac{\partial F}{\partial y}\frac{dy}{dx} = 0$$

$$ \frac{dy}{dx} = -\frac{F_x}{F_y}$$